This notebook represents a prototypical python 3 implementation for a without safeguards, without asserts, with fixed paramterized model. For larger data sets it is advised to store intermediate results, e.g. as pickle files.

Theory https://towardsdatascience.com/effortlessly-recommending-similar-images-b65aff6aabfb



# Redimensionando

Neste primeiro passo pegamos todas as imagens (.jpg) no diretório de entrada (inputDir) e as normalizamos 
para o tamanho 224x224, necessário para o PyTorch com resnet18. As imagens redimensionadas são salvas no
diretório inputDirCNN.

In [ ]:
import os
from PIL import Image
from torchvision import transforms

# needed input dimensions for the CNN
inputDim = (224,224)
inputDir = "originalImages"
inputDirCNN = "inputImagesCNN"
searchDir = "searchDir"

os.makedirs(inputDirCNN, exist_ok = True)

# clear inputDirCnn folder
for f in os.listdir(inputDirCNN):
    if not f.endswith(".jpg"):
        continue
    os.remove(os.path.join(inputDirCNN, f))



def resizeAndSaveToInputDirCNN(inputDir, imageName, inputDirCNN):
    transformationForCNNInput = transforms.Compose([transforms.Resize(inputDim)])
    
    I = Image.open(os.path.join(inputDir, imageName))
    newI = transformationForCNNInput(I)

    # copy the rotation information metadata from original image and save, else your transformed images may be rotated
    if "exif" in I.info:
        exif = I.info['exif']
        newI.save(os.path.join(inputDirCNN, imageName), exif=exif)
    else:
        newI.save(os.path.join(inputDirCNN, imageName))
    
    newI.close()
    I.close()
    return



for imageName in os.listdir(inputDir):
    resizeAndSaveToInputDirCNN(inputDir, imageName, inputDirCNN)


# Criando os vetores de características com Resnet18

Calculando os vetores de características (features) das imagens com resnet18 usando CPU (ao invés de GPU). A entrada é normalizada para os valores médios e desvio padrão da ImageNet.

In [ ]:
import torch
from tqdm import tqdm
from torchvision import models

# for this prototype we use no gpu, cuda= False and as model resnet18 to obtain feature vectors

class Img2VecResnet18():
    def __init__(self):
        
        self.device = torch.device("cpu")
        self.numberFeatures = 512
        self.modelName = "resnet-18"
        self.model, self.featureLayer = self.getFeatureLayer()
        self.model = self.model.to(self.device)
        self.model.eval()
        self.toTensor = transforms.ToTensor()
        
        # normalize the resized images as expected by resnet18
        # [0.485, 0.456, 0.406] --> normalized mean value of ImageNet, [0.229, 0.224, 0.225] std of ImageNet
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
    def getVec(self, img):
        image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device)
        embedding = torch.zeros(1, self.numberFeatures, 1, 1)

        def copyData(m, i, o): embedding.copy_(o.data)

        h = self.featureLayer.register_forward_hook(copyData)
        self.model(image)
        h.remove()

        return embedding.numpy()[0, :, 0, 0]

    def getFeatureLayer(self):
        
        cnnModel = models.resnet18(pretrained=True)
        layer = cnnModel._modules.get('avgpool')
        self.layer_output_size = 512
        
        return cnnModel, layer
        

def convertImageToFeatureVector(image, vectors):
    I = Image.open(os.path.join(inputDirCNN, image))
    vec = img2vec.getVec(I)
    vectors[image] = vec
    I.close() 
    return
        
# generate vectors for all the images in the set
img2vec = Img2VecResnet18() 

allVectors = {}
print("Converting images to feature vectors:")
for image in tqdm(os.listdir(inputDirCNN)):
    convertImageToFeatureVector(image, allVectors)


# Similaridade de cosseno (Cosine similarity)
Calculando para cada vetor a similaridade de cosseno para os outros vetores.
OBS: com milhares de imagens essa matriz pode se tornar imensa e ineficiente.

In [ ]:
%%time

# now let us define a function that calculates the cosine similarity entries in the similarity matrix
import pandas as pd
import numpy as np

def getSimilarityMatrix(vectors):
    v = np.array(list(vectors.values())).T
    sim = np.inner(v.T, v.T) / ((np.linalg.norm(v, axis=0).reshape(-1,1)) * ((np.linalg.norm(v, axis=0).reshape(-1,1)).T))
    keys = list(vectors.keys())
    matrix = pd.DataFrame(sim, columns = keys, index = keys)
    
    return matrix
        
similarityMatrix = getSimilarityMatrix(allVectors)

# print(similarityMatrix)

# Preparando as listas de top-k mais semelhantes
A partir da matriz de similaridade, armazenamos em outra estrutura de dados as k imagens mais semelhantes à cada uma do dataset de treinamento

In [ ]:
%%time

from numpy.testing import assert_almost_equal
import pickle

k = 5 # the number of top similar images to be stored

similarNames = pd.DataFrame(index = similarityMatrix.index, columns = range(k))
similarValues = pd.DataFrame(index = similarityMatrix.index, columns = range(k))

for j in tqdm(range(similarityMatrix.shape[0])):
    kSimilar = similarityMatrix.iloc[j, :].sort_values(ascending = False).head(k)
    similarNames.iloc[j, :] = list(kSimilar.index)
    similarValues.iloc[j, :] = kSimilar.values

similarNames.to_pickle("similarNames.pkl")
similarValues.to_pickle("similarValues.pkl")

# print(similarNames)

# Testando a similaridade entre as imagens de treinamento

In [ ]:
import matplotlib.pyplot as plt

# take some examples from the provided image set and plot
# inputImages = ["trees0.jpg", "camper0.jpg", "buildings0.jpg", "donkey0.jpg", "donkey_search.jpg", "kombi_search.jpg"]
# inputImages = ["20150703_0805_34.jpg" , "20150703_0825_6.jpg",
#                 "busy 20150703_0840_2.jpg", "20150703_0830_24.jpg"]
# inputImages = ["20150703_0820_39.jpg", "busy 20150703_0920_14.jpg"] # search images

inputImages = []
for f in os.listdir(inputDirCNN):
    if not f.endswith(".jpg"):
        continue
    inputImages.append(f)

numCol = 5
numRow = 1

def setAxes(ax, image, query = False, **kwargs):
    value = kwargs.get("value", None)
    if query:
        ax.set_xlabel("Query Image\n{0}".format(image[:12]), fontsize = 12, color = "red")
    else:
        ax.set_xlabel("Similarity {1:1.3f}\n{0}".format(image[:12],  value), fontsize = 12)
    ax.set_xticks([])
    ax.set_yticks([])
    
def getSimilarImages(image, simNames, simVals):
    if image in set(simNames.index):
        imgs = list(simNames.loc[image, :])
        vals = list(simVals.loc[image, :])
        if image in imgs:
            assert_almost_equal(max(vals), 1, decimal = 5)
            imgs.remove(image)
            vals.remove(max(vals))
        return imgs, vals
    else:
        print("'{}' Unknown image".format(image))
        
def plotSimilarImages(image, similarNames, similarValues):
    simImages, simValues = getSimilarImages(image, similarNames, similarValues)
    fig = plt.figure(figsize=(10, 20))
    
    # now plot the  most simliar images
    for j in range(0, numCol*numRow):
        ax = []
        if j == 0:
            if os.path.exists(os.path.join(inputDir, image)):
                img = Image.open(os.path.join(inputDir, image))
            else:
                img = Image.open(os.path.join(searchDir, image))
            ax = fig.add_subplot(numRow, numCol, 1)
            setAxes(ax, image, query = True)
        else:
            img = Image.open(os.path.join(inputDir, simImages[j-1]))
            ax.append(fig.add_subplot(numRow, numCol, j+1))
            setAxes(ax[-1], simImages[j-1], value = simValues[j-1])
        img = img.convert('RGB')
        plt.imshow(img)
        img.close()
    
    plt.show()
        
for image in inputImages:
    plotSimilarImages(image, similarNames, similarValues)
    
    
#show search image
# img = Image.open(os.path.join(searchDir, "donkey_search.jpg"))
# plt.imshow(img)
# img.close()


# Testando com uma imagem de fora do Dataset de treinamento (caso de uso mais próximo do real)

In [ ]:
%%time

def BuscarImagensSemelhantes(searchImgName):
    searchImg = Image.open(os.path.join(searchDir, searchImgName))

    # resize and save to inputDirCNN
    resizeAndSaveToInputDirCNN(searchDir, searchImgName, inputDirCNN)


    convertImageToFeatureVector(searchImgName, allVectors)


    similarityMatrix = getSimilarityMatrix(allVectors)

    # prepareTopKLists(k, similarityMatrix)
    newSimilarNames = pd.DataFrame(index = similarityMatrix.index, columns = range(k))
    newSimilarValues = pd.DataFrame(index = similarityMatrix.index, columns = range(k))

    for j in tqdm(range(similarityMatrix.shape[0])):
        kSimilar = similarityMatrix.iloc[j, :].sort_values(ascending = False).head(k)
        newSimilarNames.iloc[j, :] = list(kSimilar.index)
        newSimilarValues.iloc[j, :] = kSimilar.values

    plotSimilarImages(searchImgName, newSimilarNames, newSimilarValues)

    allVectors.pop(searchImgName) # ponto e vírgula necessário para não imprimir a saída do comando
    ; 
    return



for f in os.listdir(searchDir):
    if not f.endswith(".jpg"):
        continue
    BuscarImagensSemelhantes(f)
    
# searchImgName = "20150703_0820_39.jpg"
# #searchImgName = "busy 20150703_0920_14.jpg"
# BuscarImagensSemelhantes(searchImgName)

# total imagens: 128
# tempo tota: 54.6 s
# tempo por imagem: 0,42 s